In [ ]:
from env import DATABASE_URI
from sqlalchemy import create_engine, text, select
from sqlalchemy.orm import Session
import models as dm 
from datetime import datetime
engine = create_engine(DATABASE_URI, echo=False )


In [ ]:
THERMAL_MOCKUP_SERIAL_NUMBER = "TM1"
with Session(engine) as session:
    tm = session.execute(select(dm.Module).where(dm.Module.name == THERMAL_MOCKUP_SERIAL_NUMBER)).scalar_one_or_none()
    #tm.name = "TM2"
    #print(tm)
    #session.commit()

In [ ]:
##
#### ADD THERMAL MOCKUP
##

# increment by one for the FNAL
THERMAL_MOCKUP_SERIAL_NUMBER = "TM2"
with Session(engine) as session:
    thermal_mockup = dm.Module(
        name = THERMAL_MOCKUP_SERIAL_NUMBER
    )
    thermal_mockup.info = "Third mockup assembled at FNAL, Nov 20th 2024. Dummies: Micross 21, Micross 22, Micross 23, Micross 25"
    session.add(thermal_mockup)
    session.commit()

In [ ]:
##
#### SOLDER PLATE UPLOAD
##
with Session(engine) as session:
    # # add Cold Plate configuration (epoxy and solder v1 and solder v2)
    with open("/home/hayden/Downloads/pretty_double_loop_solder_plate.png", 'rb') as f:
        plate_image = f.read()
    plate = dm.ColdPlate(
        name='Pretty Double Loop Solder Plate',
        positions = {
            1:  'front side, top row, far left',
            2:  'front side, top row, middle left',
            3:  'front side, top row, middle right',
            4:  'front side, top row, far right',
            5:  'front side, middle row, far left',
            6:  'front side, middle row, middle left',
            7:  'front side, middle row, middle right',
            8:  'front side, middle row, far right',
            9:  'front side, bottom row, far left',
            10: 'front side, bottom row, middle left',
            11: 'front side, bottom row, middle right',
            12: 'front side, bottom row, far right',
            13: 'back side, top row, far left',
            14: 'back side, top row, middle left',
            15: 'back side, top row, middle right',
            16: 'back side, top row, far right',
            17: 'back side, middle row, far left',
            18: 'back side, middle row, middle left',
            19: 'back side, middle row, middle right',
            20: 'back side, middle row, far right',
            21: 'back side, bottom row, far left bottom',
            22: 'back side, bottom row, middle left bottom',
            23: 'back side, bottom row, middle right bottom',
            24: 'back side, bottom row, far right bottom'
        },
        plate_image = plate_image,
    )
    session.add(plate)
    session.commit()

In [ ]:
##
#### CALIBRATION DATA
## 
import json

MODULE_SERIAL_NUMBER = 'TM2'
CALIBRATION_DATA_PATH = "/home/hayden/repos/module_pcb_thermal_mockup/module_pcb_thermal_mockup/software_TM/CalibrationGUI/calibration_data_TM2_2024-12-17 14:37:49.851351.json"

ENABLED_SENSORS = ['E1','E2','E3','E4', 'L1', "L2", "L3", "L4"]

CALIBRATION_COMMENT = "This the first calibration done in the TIF Climate Box: Batiment 28 2-020"

with Session(engine) as session:
    query = select(dm.Module).where(dm.Module.name == MODULE_SERIAL_NUMBER)
    thermal_mockup = session.execute(query).scalars().first()
    
    with open(CALIBRATION_DATA_PATH, 'r') as f:
        calib_data = json.load(f)

    db_calibs = []
    for sensor in ENABLED_SENSORS:
        db_calibs.append(
            dm.SensorCalibration(
                module = thermal_mockup,
                sensor = sensor,
                fit_ohms_over_celcius = calib_data[sensor]['fit_slope'],
                fit_ohms_intercept = calib_data[sensor]['fit_intercept'],
                celcius = calib_data[sensor]['temps'],
                ohms = calib_data[sensor]['ohms'],
                raw_adc = calib_data[sensor]['raw_adcs'],
                times = calib_data[sensor]['times'],
                all_raw_adcs = calib_data[sensor]['all_raw_adcs'],
                all_raw_times = calib_data[sensor]['all_times'],
            )
        )
    session.add_all(db_calibs)
    mod_recalib = dm.ModuleCalibration(
        E1 = db_calibs[0],
        E2 = db_calibs[1],
        E3 = db_calibs[2],
        E4 = db_calibs[3],
        L1 = db_calibs[4],
        L2 = db_calibs[5],
        L3 = db_calibs[6],
        L4 = db_calibs[7],
        module = thermal_mockup,
        comment=CALIBRATION_COMMENT
    )

    session.add(mod_recalib)

    thermal_mockup.calibration = mod_recalib

    session.commit()

In [ ]:
# query calibration
with Session(engine) as session:
    module = session.execute(select(dm.Module).where(dm.Module.name=="TM1")).scalar_one_or_none()
    print(module.calibration)

In [ ]:
with Session(engine) as session:
    module = select(dm.Module).where(dm.Module.name=="TM0002")
    module.info = "Second thermal mockup, assembled at FNAL Nov 20th, 2024. Dummies: Micross 15, Micross 16, Micross 17, Micross 18"
    session.commit()

In [ ]:
# query runs
with Session(engine) as session:
    for run in session.execute(select(dm.Run)):
        print(run)

In [ ]:
from pydantic import BaseModel, field_validator, ConfigDict, model_validator, model_serializer
from typing import Optional
from functools import partial

# def db_exists_validator(cls, value, db_model, column):
#     if isinstance(value, str):
#         # Query for module
#         obj = cls.session.execute(
#             select(db_model).where(column == value)
#         ).scalar()

#         assert obj is not None, f"{db_model.__name__} not found in database"
#         return obj
#     return value


class DBBaseModel(BaseModel):

    @classmethod
    def set_session(cls, session):
        cls.session = session

    @classmethod
    def exists_validator(cls, value, db_model, column):
        if isinstance(value, str):
            # Query for module
            obj = cls.session.execute(
                select(db_model).where(column == value)
            ).scalar()

            assert obj is not None, f"{db_model.__name__} not found in database"
            return obj
        return value

class ModuleConfig(DBBaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    mod: dm.Module
    
    _mod_exists_validator = field_validator('mod', mode='before')(
        partial(DBBaseModel.exists_validator, db_model=dm.Module, column=dm.Module.name)
    )
    # @field_validator('mod', mode='before')
    # @classmethod
    # def validate_mod(cls, v: str) -> dm.Module:
    #     if isinstance(v, str):
    #         # Query for module
    #         module = cls.session.execute(
    #             select(dm.Module).where(dm.Module.name == v)
    #         ).scalar()

    #         assert module is not None, "Module not found in database"
    #         return module
    #     return v



# Example usage

DBBaseModel.set_session(session)
module_instance = ModuleConfig(mod="TM0001")
print(module_instance.mod)

print(module_instance)

In [ ]:
with Session(engine) as session:
    module = session.execute(select(dm.Module).where(dm.Module.name == "TM0")).scalar()
    print(module.calibration)